In [21]:
import autogen
from autogen import Agent, AssistantAgent, ConversableAgent, UserProxyAgent
#from autogen.agentchat.contrib.llava_agent import LLaVAAgent, llava_call
import os
import cv2
import matplotlib.pyplot as plt

import os
from dotenv import load_dotenv

In [22]:
load_dotenv()

gemini_api_key = os.getenv('GEMINI_API_KEY')
os.environ['GEMINI_API_KEY'] = gemini_api_key

In [23]:
# gpt4_llm_config =[{
#         "model": "gpt-4",
#         "api_key": os.environ['OPENAI_API_KEY'],
#        # "tags": ["gpt-4", "tool"]
#     }]

In [24]:
gemini_llm_config = [{
        "model": "gemini-pro",
        "api_key": os.environ['GEMINI_API_KEY'],
        "api_type": "google"
    }]

In [25]:
#LLAVA_MODE = "remote"  # Either "local" or "remote"
#assert LLAVA_MODE in ["local", "remote"]

In [26]:
# from dotenv import load_dotenv
# import os

In [27]:
# load_dotenv()
# replicate_key = os.getenv("REPLICATE_API_TOKEN")

In [28]:
# replicate_key = 'r8_58e8FEvG22PWEIXQlJVNy0S0jj76yb50jWO8R'

In [29]:
# import os
# os.environ["REPLICATE_API_TOKEN"] = replicate_key

In [30]:
# os.environ["REPLICATE_API_TOKEN"]

In [31]:
# if LLAVA_MODE == "remote":

#     llava_config_list = [
#         {
#             "model": "whatever, will be ignored for remote",  # The model name doesn't matter here right now.
#             "api_key": os.environ["REPLICATE_API_TOKEN"],
#             "base_url": "yorickvp/llava-13b:2facb4a474a0462c15041b78b1ad70952ea46b5ec6ad29583c0b29dbd4249591",
#             #'base_url': "https://api.replicate.com/v1/predictions",
#             #'base_url': "https://replicate.com/yorickvp/llava-13b:2facb4a474a0462c15041b78b1ad70952ea46b5ec6ad29583c0b29dbd4249591"
#         }
#     ]

In [32]:
# image_agent = LLaVAAgent(
#     name="image-explainer",
#     max_consecutive_auto_reply=10,
#     llm_config={"config_list":  [config.copy() for config in gpt4_llm_config], "temperature": 0.5, "max_new_tokens": 1000},
# )

# user_proxy = autogen.UserProxyAgent(
#     name="User_proxy",
#     system_message="A human admin.",
#     code_execution_config={
#         "last_n_messages": 3,
#         "work_dir": "groupchat",
#         "use_docker": False,
#     },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
#     human_input_mode="NEVER",  # Try between ALWAYS or NEVER
#     max_consecutive_auto_reply=0,
# )

# user_proxy.initiate_chat(
#     image_agent,
#     message="""What's the breed of this dog?
# <img https://th.bing.com/th/id/R.422068ce8af4e15b0634fe2540adea7a?rik=y4OcXBE%2fqutDOw&pid=ImgRaw&r=0>.""",
# )

In [33]:
def resize_to_fit(image, max_height, max_width):
    """
    Resize an image to fit within the given dimensions while maintaining aspect ratio.

    Parameters:
    image (np.ndarray): The image to resize.
    max_height (int): The maximum height of the resized image.
    max_width (int): The maximum width of the resized image.

    Returns:
    np.ndarray: The resized image.
    """
    h, w = image.shape[:2]
    if h > max_height or w > max_width:
        scaling_factor = min(max_height / h, max_width / w)
        new_size = (int(w * scaling_factor), int(h * scaling_factor))
        return cv2.resize(image, new_size, interpolation=cv2.INTER_AREA)
    return image

def combine_images_grid(base_image, overlays):
    """
    Combine multiple overlay images into the base image using a 3x3 grid system.

    Parameters:
    base_image (np.ndarray): The base image where overlay images will be placed.
    overlays (list of tuples): List of overlay images and their grid positions. Each tuple should be in the format
                               (overlay_image, (grid_row, grid_col)), where (grid_row, grid_col) specifies the grid cell.

    Returns:
    np.ndarray: The combined image with overlays.
    """
    # Make a copy of the base image to modify
    combined_image = base_image.copy()

    # Get the dimensions of the base image
    base_h, base_w = combined_image.shape[:2]

    # Calculate the size of each grid cell
    cell_h, cell_w = base_h // 3, base_w // 3

    # Iterate through the overlay images and their grid positions
    for overlay, (grid_row, grid_col) in overlays:
        # Resize the overlay to fit within the grid cell
        overlay = resize_to_fit(overlay, cell_h, cell_w)

        # Get the dimensions of the overlay image
        overlay_h, overlay_w = overlay.shape[:2]

        # Calculate the top-left corner of the grid cell
        x = grid_col * cell_w
        y = grid_row * cell_h

        # Place the overlay image in the center of the grid cell
        center_x = x + (cell_w - overlay_w) // 2
        center_y = y + (cell_h - overlay_h) // 2

        # Ensure the region is within the bounds of the base image
        if center_y + overlay_h > base_h or center_x + overlay_w > base_w:
            raise ValueError("Overlay image exceeds the bounds of the base image at the specified location.")

        # Place the overlay image on the base image
        combined_image[center_y:center_y+overlay_h, center_x:center_x+overlay_w] = overlay

    return combined_image


In [34]:
# image_assistant = autogen.AssistantAgent(
#     name="image_assistant",
#     llm_config=llava_config_list[0],
#     system_message = """
#     You first get an input from the message. You then take that and pass that to combine_images_grid function defined before.
#     You then return the combined image. You will pass that to our critic agent.
#     """
# )

In [35]:
# critic_agent = autogen.AssistantAgent(
#     name="Critic",
#     llm_config=llava_config_list[0],
#     system_message="""
#     You are an expert in advertisement design and know what constitutes the best ads. Based on the recommended positions for each component, please analyze the provided image and offer feedback on where each component should be placed according to the guidelines below:

#     VERTICAL_POSITIONING = {'Logo': [1], 'CTA Button': [1, 2, 3], 'Icon': [1, 2, 3], 'Product Image': [2],
#                'Text Elements': [1,3], 'Infographic': [2], 'Banner': [1], 'Illustration': [2], 'Photograph': [2],
#                'Mascot': [2], 'Testimonial Quotes': [2], 'Social Proof': [2, 1, 3], 'Seal or Badge': [3, 1, 2],
#                'Graphs and Charts': [2], 'Decorative Elements': [3], 'Interactive Elements': [2],
#                'Animation': [2], 'Coupon or Offer Code': [3], 'Legal Disclaimers or Terms': [3],
#                'Contact Information': [3, 1, 2], 'Map or Location Image': [3], 'QR Code': [3, 1, 2]}

#     HORIZONTAL_POSITIONING = {'Logo': [1], 'CTA Button': [2, 1, 3], 'Icon': [1], 'Product Image': [1],
#                           'Text Elements': [1], 'Infographic': [1], 'Banner': [2], 'Illustration': [2],
#                           'Photograph': [2], 'Mascot': [1], 'Testimonial Quotes': [2], 'Social Proof': [3, 1, 2],
#                           'Seal or Badge': [3, 1, 2], 'Graphs and Charts': [1], 'Decorative Elements': [3],
#                           'Interactive Elements': [2], 'Animation': [2], 'Coupon or Offer Code': [3],
#                           'Legal Disclaimers or Terms': [3], 'Contact Information': [3, 1, 2],
#                           'Map or Location Image': [3], 'QR Code': [3, 1, 2]}


#     Instructions:

#     1. Identify each component in the provided image.
#     2. Provide feedback on where each component should be placed based on the vertical and horizontal positioning guidelines. For example, "Move the logo to the top right if it's not there."
#     3. Instead of providing the feedback in text, can you provide the region where each component should be. Only givr feedback to identified one.
#     """
# )

In [36]:
car_image_path = '../data/0a59be2e7dd53d6de11a10ce3649c081/game_endframe.png'
base_image_path = '../data/0a59be2e7dd53d6de11a10ce3649c081/landing_1.png'
cta_image_path = '../data/0a59be2e7dd53d6de11a10ce3649c081/cta.png'

# Load the images using cv2.imread
base_image = cv2.imread(base_image_path)
cta_image = cv2.imread(cta_image_path)
car_image = cv2.imread(car_image_path)

print(type(base_image))

# Check the loaded images dimensions
print(f"Base image shape: {base_image.shape}")
print(f"CTA image shape: {cta_image.shape}")
print(f"Car image shape: {car_image.shape}")

# Define the overlays with their grid positions
overlays = [
    (cta_image, (0, 2)),
    (car_image, (1, 1)) 
]

# Combine the images using the grid system with text extraction
combined_image = combine_images_grid(base_image, overlays)

<class 'numpy.ndarray'>
Base image shape: (900, 600, 3)
CTA image shape: (64, 300, 3)
Car image shape: (119, 224, 3)


In [38]:
image_tasks = {
    'base_image': base_image,
    'overlays': overlays
}

In [39]:
type(image_tasks['base_image'])

numpy.ndarray

In [40]:
type(image_tasks['overlays'])

list

In [ ]:
user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    human_input_mode="NEVER",
    #is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "groupchat",
        "use_docker": False,
    },
)

image_assistant = autogen.AssistantAgent(
    name="image_compose",
    system_message = """
    You first get an input from the message. You then take that and pass that to combine_images_grid function defined before.
    You then return the combined image. You will pass that to our critic agent.
    """,
    llm_config={
        "temperature": 0,
        "timeout": 600,
        "cache_seed": 42,
        "config_list": [config.copy() for config in gemini_llm_config],
        "functions": [
             {
                        "name": "combine_images_grid",
                        "description": "Ask the function to compose images based on the inputs.",
                        "parameters": {
                            "type": "object",
                            "properties": {
                                "base_image": {
                                    "type": "array",
                                    "description": "The background image in which the composition of the other images will be upon."
                                },
                                "overlays": {
                                    "type": "array",
                                    "items": {
                                        "type": "object",
                                        "properties": {
                                            "overlay_image": {
                                                "type": "array",
                                                "description": "The overlay image to place on the base image."
                                            },
                                            "grid_position": {
                                                "type": "array",
                                                "items": {"type": "integer"},
                                                "description": "The grid position (row, col) where the overlay image will be placed."
                                            }
                                        }
                                    },
                                    "description": "A list with the image and their respective region in the background image."
                                }
                            },
                            "required": ["base_image", "overlays"]
                        }
              }
        ],

    },
)



critic = autogen.AssistantAgent(
    name="Critic",
    system_message=""" You are an expert in advertisement design and know what constitutes the best ads. Based on the recommended positions for each component, please analyze the provided image and offer feedback on where each component should be placed according to the guidelines below:

    VERTICAL_POSITIONING = {'Logo': [1], 'CTA Button': [1, 2, 3], 'Icon': [1, 2, 3], 'Product Image': [2],
               'Text Elements': [1,3], 'Infographic': [2], 'Banner': [1], 'Illustration': [2], 'Photograph': [2],
               'Mascot': [2], 'Testimonial Quotes': [2], 'Social Proof': [2, 1, 3], 'Seal or Badge': [3, 1, 2],
               'Graphs and Charts': [2], 'Decorative Elements': [3], 'Interactive Elements': [2],
               'Animation': [2], 'Coupon or Offer Code': [3], 'Legal Disclaimers or Terms': [3],
               'Contact Information': [3, 1, 2], 'Map or Location Image': [3], 'QR Code': [3, 1, 2]}

    HORIZONTAL_POSITIONING = {'Logo': [1], 'CTA Button': [2, 1, 3], 'Icon': [1], 'Product Image': [1],
                          'Text Elements': [1], 'Infographic': [1], 'Banner': [2], 'Illustration': [2],
                          'Photograph': [2], 'Mascot': [1], 'Testimonial Quotes': [2], 'Social Proof': [3, 1, 2],
                          'Seal or Badge': [3, 1, 2], 'Graphs and Charts': [1], 'Decorative Elements': [3],
                          'Interactive Elements': [2], 'Animation': [2], 'Coupon or Offer Code': [3],
                          'Legal Disclaimers or Terms': [3], 'Contact Information': [3, 1, 2],
                          'Map or Location Image': [3], 'QR Code': [3, 1, 2]}


    Instructions:

    1. Identify each component in the provided image.
    2. Provide feedback on where each component should be placed based on the vertical and horizontal positioning guidelines. For example, "Move the logo to the top right if it's not there."
    3. Instead of providing the feedback in text, can you provide the region where each component should be. Only givr feedback to identified one.
    """,
    llm_config={"config_list": [config.copy() for config in gemini_llm_config]},
)

groupchat_1 = autogen.GroupChat(agents=[user_proxy, image_assistant, critic], messages=[], max_round=50)

manager_1 = autogen.GroupChatManager(
    groupchat=groupchat_1,
    name="Research_manager",
    llm_config={"config_list": [config.copy() for config in gemini_llm_config]},
    #is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "groupchat",
        "use_docker": False,
    },
)


user = autogen.UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    #is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "tasks",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)
user.initiate_chats(
    [
        {"recipient": image_assistant, "message": f"The input for combine_images_grid is in {image_tasks}", "summary_method": "last_msg"},
    ]
)


********************************************************************************
Starting a new chat....

********************************************************************************
User (to Researcher):

The input for combine_images_grid is in {'base_image': array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [25